In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bigmart-sales-data/Train.csv
/kaggle/input/bigmart-sales-data/Test.csv


#### **Data Science Project : Predicting Big Mart Sales using Machine Learning Techniques**

Big Mart is one stop shopping center and free marketplace.
We also wholesale fashion accessories, gold or silver plated chain necklaces,
bodyjewelry, sunglasses, wallets, scarves, and belts.

The objectif of this project is to predict the outcome of sales of the big mart grocery stores.

**Step 1: Importing the Data**

In [2]:
### Importing the Useful libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Importing ML Libraries
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn import metrics
%matplotlib inline

In [3]:
raw_data = pd.read_csv("../input/bigmart-sales-data/Train.csv")

In [4]:
## Copying the DATA to keep the original one and work with the copy to analyse the data
df = raw_data.copy()

In [5]:
display("The Shape of the DataSet is {}".format(df.shape), df.head(10))

'The Shape of the DataSet is (8523, 12)'

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [6]:
# Exploring the Dataset's columns
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

Comment : Based on the result above, we have 5 numeric columns and 7 object type features. 
We will now explore deeply every single column to see the unique values of each of them, in order to better understand the dataset to lead a better data analysis.

In [7]:
# Displaying the Unique Values of each columns
for column in df:
  if len(df[column].unique()) < 10:
    print('The Number of Values for the Feature {} : {} ---- {}'.format(column, 
                                                                        df[column].unique(),
                                                                        len(df[column].unique()),
                                                                       type(df[column])))
  else:
    print('The Number of values for the Feature {} : ---- {}'.format(column, len(df[column].unique())))

The Number of values for the Feature Item_Identifier : ---- 1559
The Number of values for the Feature Item_Weight : ---- 416
The Number of Values for the Feature Item_Fat_Content : ['Low Fat' 'Regular' 'low fat' 'LF' 'reg'] ---- 5
The Number of values for the Feature Item_Visibility : ---- 7880
The Number of values for the Feature Item_Type : ---- 16
The Number of values for the Feature Item_MRP : ---- 5938
The Number of values for the Feature Outlet_Identifier : ---- 10
The Number of Values for the Feature Outlet_Establishment_Year : [1999 2009 1998 1987 1985 2002 2007 1997 2004] ---- 9
The Number of Values for the Feature Outlet_Size : ['Medium' nan 'High' 'Small'] ---- 4
The Number of Values for the Feature Outlet_Location_Type : ['Tier 1' 'Tier 3' 'Tier 2'] ---- 3
The Number of Values for the Feature Outlet_Type : ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3'] ---- 4
The Number of values for the Feature Item_Outlet_Sales : ---- 3493


Comment : With the code above we can see that in the original Dataset, there was an error in the *Item_Fat_Content* column.
Hence we have 2 different values that designate the same one (Low Fat and LF) and (Regular and reg),
So we will replace (LF to Low Fat) and (reg to Regular).


In [8]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'})

In [9]:
## Rechecking for the changements using unique values code
for column in df:
  if len(df[column].unique()) < 10:
    print('The Number of Values for the Feature {} : {} ---- {}'.format(column, 
                                                                        df[column].unique(),
                                                                        len(df[column].unique()),
                                                                       type(df[column])))
  else:
    print('The Number of values for the Feature {} : ---- {}'.format(column, len(df[column].unique())))

The Number of values for the Feature Item_Identifier : ---- 1559
The Number of values for the Feature Item_Weight : ---- 416
The Number of Values for the Feature Item_Fat_Content : ['Low Fat' 'Regular'] ---- 2
The Number of values for the Feature Item_Visibility : ---- 7880
The Number of values for the Feature Item_Type : ---- 16
The Number of values for the Feature Item_MRP : ---- 5938
The Number of values for the Feature Outlet_Identifier : ---- 10
The Number of Values for the Feature Outlet_Establishment_Year : [1999 2009 1998 1987 1985 2002 2007 1997 2004] ---- 9
The Number of Values for the Feature Outlet_Size : ['Medium' nan 'High' 'Small'] ---- 4
The Number of Values for the Feature Outlet_Location_Type : ['Tier 1' 'Tier 3' 'Tier 2'] ---- 3
The Number of Values for the Feature Outlet_Type : ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3'] ---- 4
The Number of values for the Feature Item_Outlet_Sales : ---- 3493


In [10]:
# Analysing Statistically the dataset features
display(df.describe(),
        df.describe(include = object))

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,2,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5517,1232,935,2793,3350,5577


In [11]:
df.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000


In [12]:
### Checking for the Kurtosis and the Skewness of the Dataset
# Displaying the Kurtosis and the Skewness of the Data
display("The Kurtosis of the DataSet is {}".format(df.kurtosis()))

display("The Skewness of the DataSet is {}".format(df.skew()))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


'The Kurtosis of the DataSet is Item_Weight                 -1.227766\nItem_Visibility              1.679445\nItem_MRP                    -0.889769\nOutlet_Establishment_Year   -1.205694\nItem_Outlet_Sales            1.615877\ndtype: float64'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """


'The Skewness of the DataSet is Item_Weight                  0.082426\nItem_Visibility              1.167091\nItem_MRP                     0.127202\nOutlet_Establishment_Year   -0.396641\nItem_Outlet_Sales            1.177531\ndtype: float64'

In [13]:
## Checking for Duplicate rows to drop them 
### Checking for the DataSet shape before removing the Data
print("The Shape of the Data Before removing duplicate rows is {}".format(df.shape))

### The Code to remove the duplicate rows
duplicate_rows_df = df[df.duplicated()]

### Displaying the Duplicate rows DataFrame
display("The Shape of the Duplicate Rows DataFrame is {}".format(duplicate_rows_df.shape))

The Shape of the Data Before removing duplicate rows is (8523, 12)


'The Shape of the Duplicate Rows DataFrame is (0, 12)'

#### Now let's Check for the missing Data


In [14]:
display("The Number of rows in the ".df.shape[0], df.isnull().sum())

AttributeError: 'str' object has no attribute 'df'

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Missing Data'})
plt.savefig("visualizing_missing_data_with_heatmap_Seaborn_Python.png", dpi=100)

In [ ]:
# Using DataViz to better view the missing values on the dataset
import seaborn as sns
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(df.isnull(), cbar = False, yticklabels = False, vmin=0.5, vmax=0.7, ax = ax)

In [ ]:
plt.figure(figsize=(10,6))
sns.displot(
    data=df.isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill",
    aspect=1.25
)
plt.savefig("visualizing_missing_data_with_barplot_Seaborn_distplot.png", dpi=100)

**Comment :** As we can see in the graph above and the one 

#### Python | Visualize missing values (NaN) values using Missingno Library

Missingno library offers a very nice way to visualize the distribution of NaN values. Missingno is a Python library and compatible with Pandas.

In [ ]:
pip install missingno

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df)

In [ ]:
msno.bar(df)

In [ ]:
## Visualizing the missing data using heatmaps

# Also, missingno.heatmap visualizes the correlation matrix about the locations of missing values in columns.

msno.heatmap(df)

In [ ]:
### Styding the Correlation between variables using scatter plots

# We will begin by selecting only the numeric columns
num_columns = df.select_dtypes(include=np.number).columns.tolist()
num_columns.append('Item_Fat_Content')
num_columns

In [ ]:
sns.pairplot(df[num_columns], hue = 'Item_Fat_Content')

In [ ]:
for column in df[num_columns]:
    display(df.hist(column = column, grid = False, color='firebrick', bins = 3))

In [ ]:
# filling the missing values in "Item_weight column" with "Mean" value
df['Item_Weight'].fillna(df['Item_Weight'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df['Item_Type'].value_counts

In [ ]:
## Putting Columns with less than 10 unique values together 

## Rechecking for the changements using unique values code

columns_with_less_than_10_unique_values = []

for column in df:
  if len(df[column].unique()) < 10:
    columns_with_less_than_10_unique_values.append(column)
    

In [ ]:
print(columns_with_less_than_10_unique_values)

In [ ]:
for col in columns_with_less_than_10_unique_values:
    print(type(col))

In [ ]:
for col in columns_with_less_than_10_unique_values:
    plt.figure(figsize=(8.8, 8.8))
    sns.distplot(list(df[col]))
    plt.show()

In [ ]:
for col in columns_with_less_than_10_unique_values:
    plt.figure(figsize=(8.8, 8.8))
    sns.countplot(x=col, data=df, hue = 'Outlet_Size')
    plt.show()

In [ ]:
## Label Encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df['Item_Identifier'] = encoder.fit_transform(df['Item_Identifier'])

df['Item_Fat_Content'] = encoder.fit_transform(df['Item_Fat_Content'])

df['Item_Type'] = encoder.fit_transform(df['Item_Type'])

df['Outlet_Identifier'] = encoder.fit_transform(df['Outlet_Identifier'])

df['Outlet_Size'] = encoder.fit_transform(df['Outlet_Size'])

df['Outlet_Location_Type'] = encoder.fit_transform(df['Outlet_Location_Type'])

df['Outlet_Type'] = encoder.fit_transform(df['Outlet_Type'])

In [ ]:
df.head()

In [ ]:
X = df.drop(columns='Item_Outlet_Sales', axis=1)
y = df['Item_Outlet_Sales']

In [ ]:
display(type(X), type(y))

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

#### Training the ML Model

XGBoost Regressor

In [ ]:
regressor = XGBRegressor()

In [ ]:
regressor.fit(X_train, Y_train)

In [ ]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

In [ ]:
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [ ]:
print('R Squared value = ', r2_train)

In [ ]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)

In [ ]:
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)

In [ ]:
print('R Squared value = ', r2_test)